In [1]:
from ExpectationMaximization import ExpectationMaximization
import numpy as np
import matplotlib.pyplot as plt
from CarGenerator import CarGenerator, Direction
from scipy.stats import mode

In [2]:
kitti_path = '/scratch/local/hdd/hizkia/kitti'
date = '2011_09_29'
reference_rectangle = (64, 128)
min_original_rectangle = (32, 64)

In [3]:
def normalize_depth(depth, instance):
    return np.multiply(depth, instance == mode(instance.ravel())[0][0]) / np.median(depth[instance == mode(instance.ravel())[0][0]])

In [3]:
em = ExpectationMaximization(kitti_path,
                             date=date,
                             reference_rectangle=reference_rectangle,
                             min_original_rectangle=min_original_rectangle,
                             max_iters = 5
                            )

In [4]:
em.run()

Generating samples...
Initializing EM yields a score of 113402769.16251183...
EM stopped with a score of 113402769.16251183


In [4]:
generator = CarGenerator(kitti_path, date=date, reference_rectangle=reference_rectangle, min_original_rectangle=min_original_rectangle, depth_normalization_func=normalize_depth).load_dataset()

In [5]:
def calculate_EM_score(normalized_samples, cov):
    inv_cov = np.linalg.inv(cov)
    return sum(np.apply_along_axis(lambda x: x.T @ inv_cov @ x, 0, normalized_samples))

def _help(i, x):
    print(i,end='\r')
    return x.normalized_depth.ravel()

In [6]:
D = np.array([_help(i,x) for i,x in enumerate(generator)]).T

In [8]:
mean = D.mean(axis = 1)
A = D - mean.reshape((-1,1))
cov = np.cov(A)
if not np.all(np.linalg.eigvals(cov) > 0):
    cov += np.eye(cov.shape[0])

score = calculate_EM_score(A, cov)

In [9]:
score

115656.0273577545

In [12]:
generator = CarGenerator(kitti_path, date=date, reference_rectangle=reference_rectangle, min_original_rectangle=min_original_rectangle, depth_normalization_func=normalize_depth, mean=mean, cov=cov, optimize_direction=Direction.HORIZONTAL).load_dataset()

In [13]:
temp_D = np.array([_help(i,x) for i,x in enumerate(generator)]).T

In [14]:
temp_mean = D.mean(axis=1)
temp_A = temp_D - temp_mean.reshape((-1,1))
temp_cov = np.cov(temp_A)
if not np.all(np.linalg.eigvals(temp_cov) > 0):
    temp_cov += np.eye(temp_cov.shape[0])

temp_score = calculate_EM_score(temp_A, temp_cov)

In [15]:
calculate_EM_score(temp_A, temp_cov)

115067.72766368101

In [20]:
calculate_EM_score(D-temp_mean.reshape((-1,1)), np.eye(8192))

1208301.7541550347

In [21]:
calculate_EM_score(A, np.eye(8192))

1225069.108359625